In [ ]:
import face_recognition
import numpy as np
import pandas as pd
import glob
import shutil

# Carrega a imagem de referência do rosto da pessoa
person_image = face_recognition.load_image_file('FACE.JPG')
person_face_encoding = face_recognition.face_encodings(person_image)[0]

# Cria uma lista com o face encoding e nomes
known_face_encodings = [
    person_face_encoding
]
known_face_names = [
    'Rodrigo Cunha'
]
print('Learned encoding for', len(known_face_encodings), 'images.')

In [ ]:
#Cria lista com todos os arquivos a serem analisados
files_list = list(glob.iglob('E:\\Rodrigo\\RECONHECIMENTO FACIAL\\FOTOS\\FOTOS\\*.JPG', recursive = True))

#Transforma a lista em dataframe
df_files = pd.DataFrame(files_list, columns = ['filename'])
df_files['known_faces'] = np.nan
df_files

In [ ]:
#Compara todos os rostos em cada um dos arquivos com os rostos na lista known_face_encodings
for index, file in df_files[df_files['known_faces'].isna()].iterrows():
    print(file['filename'])
    # Carrega a imagem com os rostos desconhecidos
    unknown_image = face_recognition.load_image_file(file['filename'])

    # Localiza todos os rostos e face encodings na imagem
    face_locations = face_recognition.face_locations(unknown_image)
    face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

    # Realiza loop em cada rosto na imagem
    for face_encoding in face_encodings:
        # Compara se o rosto combina com algum dos rostos conhecidos na lista known_face_encodings
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        name = "Unknown"

        # Ou compara a face_distance dos rostos conhecidos
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        # Se o rosto for identificado, altera o dataframe e copia o arquivo para outra pasta
        if(name in known_face_names):
            df_files.at[index,'known_faces'] = name
            n_filename = file['filename'].split('\\')
            n_filename = n_filename[-1]
            shutil.copyfile(file['filename'], f'E:\\Rodrigo\\RECONHECIMENTO FACIAL\\FOTOS\\FOTOS_FILT\\{n_filename}')
            print(f'O arquivo {n_filename} foi copiado')
            break
    
    # Se o rosto não for identificado, insere False na colua 'known_faces'
    df_files.at[index,'known_faces'] = False


In [ ]:
# Salva o dataframe df_files em arquivo
df_files.to_csv('files.csv', sep=';', encoding='Windows-1252', index=False)
df_files